In [1]:
%pip install --upgrade groq python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# try to get env variables
import os
os.environ.get('GROQ_API_KEY')

'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'

In [4]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')

# Create a cursor object
cursor = conn.cursor()

# Query all the pages
cursor.execute('SELECT * FROM blocks')
blocks = cursor.fetchall()
print('Total blocks: ', len(blocks))

Total blocks:  15398


In [5]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')

# Create a cursor object
cursor = conn.cursor()
# Query all the pages
cursor.execute('SELECT * FROM pages')
pages = cursor.fetchall()
print('Total pages: ', len(pages))

Total pages:  356


In [43]:
import sqlite3
import pickle

# Connect to the database
conn = sqlite3.connect('data.db')

# Create a cursor object
cursor = conn.cursor()

# Create a cursor object
cursor = conn.cursor()

# Query the embeddings from the database
cursor.execute('SELECT * FROM embeddings')
retrieved_embeddings = cursor.fetchall()

# Deserialize the embeddings
retrieved_embeddings = [pickle.loads(row[1]) for row in retrieved_embeddings]

print('Shape of retrieved embeddings: ', len(retrieved_embeddings), 'x', len(retrieved_embeddings[0]))


Shape of retrieved embeddings:  1 x 15778


In [34]:
print("Shape of retrieved embeddings: ", len(retrieved_embeddings), 'x', len(retrieved_embeddings[0]))

Shape of retrieved embeddings:  1 x 15778


In [35]:
from fastembed import TextEmbedding
# This will trigger the model download and initialization
embedding_model = TextEmbedding()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [47]:
query = [ 'What is the purpose of life?' ]
# This will trigger the model download and initialization
embedding_model = TextEmbedding()
query_embedding = list(embedding_model.embed(query))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [48]:
from sklearn.metrics.pairwise import cosine_similarity
# Join the blocks embeddings with the pages embeddings
combined_embeddings = retrieved_embeddings[0]
similarities = cosine_similarity(query_embedding, combined_embeddings)
print("Shape of similarities: ", similarities.shape)

Shape of similarities:  (1, 15778)


In [49]:
import numpy as np
import pandas as pd

blocks_length = 15066
pages_length = 356

def find_most_similar_embeddings(similarities, blocks, pages, blocks_length, pages_length, top_n=10):
    # Find the most similar embedding
    top_5_indices = np.argsort(similarities[0])[-top_n:][::-1]
    top_data = [
        ("block", blocks[i][3], blocks[i][2], similarities[0, i]) if i < blocks_length 
        else ("page", pages[i - blocks_length][1], pages[i - blocks_length][2], similarities[0, i]) 
        if i < blocks_length + pages_length
        else ("page_title", pages[i - blocks_length - pages_length][1], pages[i - blocks_length - pages_length][2], similarities[0, i])
        for i in top_5_indices
    ]
    return top_data

top_data = find_most_similar_embeddings(similarities, blocks, pages, blocks_length, pages_length)
pd.set_option('display.max_colwidth', 100)
pd.DataFrame(top_data, columns=["Type", "Content", "Path", "Score"])

,Type,Content,Path,Score
0,block,\t\t\t- Our purpose is to glorify God and find joy in him,/Users/freddycaceres/Research/Second brain/2022_12_25.md#9,0.721803
1,block,\t- The purpose driven life,/Users/freddycaceres/Research/Second brain/2023_05_13.md#3,0.721707
2,block,\t\t- **Tags**: #[[favorite]],"/Users/freddycaceres/Research/Second brain/My Early Life - Winston S. Churchill, William Manches...",0.720421
3,block,"\t- Rhetoric falls into three divisions, determined by the three classes of listeners to speeche...",/Users/freddycaceres/Research/Second brain/Rhetoric - Aristotle.md#27,0.705235
4,block,"last_highlighted_date:: December 17, 2022",/Users/freddycaceres/Research/Second brain/The Fastest Way to Kill... - @Austen on Twitter.md#5,0.693172
5,block,published_date::,/Users/freddycaceres/Research/Second brain/Tweets From Figen - @TheFigen on Twitter.md#7,0.692874
6,block,\t- I often talk about how I want to live a purpose driven life but in the last years I have bee...,/Users/freddycaceres/Research/Second brain/2024_07_09.md#19,0.684423
7,block,\t },/Users/freddycaceres/Research/Second brain/to___study.md#124,0.678940
8,block,\t (sort-by (fn [h],/Users/freddycaceres/Research/Second brain/to___study.md#122,0.670128
9,block,"\t- I was once asked to devise an inscription for a monument in France. I wrote, ‘In war, Resolu...","/Users/freddycaceres/Research/Second brain/My Early Life - Winston S. Churchill, William Manches...",0.669704


In [12]:
top_data

[('block',
  '- Great quote from [[persons/Benjamin Franklin]] about a happy meaningful life, and the art of journaling that life',
  '/Users/freddycaceres/Research/Second brain/2023_06_28.md#0',
  0.76174617),
 ('block',
  '\t- Of the joy he found in writing, he said, ‘To sit at one’s table on a sunny morning, with four clear hours of uninterruptible security, plenty of nice white paper, and a Squeezer pen*\xa0– that is true happiness.’ ([Location\xa02998](https://readwise.io/to_kindle?action=open&asin=B079R3VH13&location=2998))',
  '/Users/freddycaceres/Research/Second brain/Churchill - Andrew Roberts.md#114',
  0.7252633)]

In [13]:
# matched_info = ' '.join(item['metadata']['text'] for item in result['matches'])
# sources = [item['metadata']['source'] for item in result['matches']]
# generate a test context based on Franklin's writing
block_1 = """
{
title: The Autobiography of Benjamin Franklin - Benjamin Franklin
author: Benjamin Franklin
text: That felicity, when I reflected on it, has induced me sometimes to say, that were it offered to my choice, I should have no objection to a repetition of the same life from its beginning, only asking the advantages authors have in a second edition to correct some faults of the first. So I might, besides correcting the faults, change some sinister accidents and events of it for others more favorable. But though this were denied, I should still accept the offer. Since such a repetition is not to be expected, the next thing most like living one’s life over again seems to be a recollection of that life, and to make that recollection as durable as possible by putting it down in writing.
}
""" 
block_2 = """
{
title: My Early Life - Winston S. Churchill, William Manchester (Introduction)
author: Winston Churchill
text: Where my reason, imagination or interest were not engaged, I would not or I could not learn. In all the twelve years I was at school no one ever succeeded in making me write a Latin verse or learn any Greek except the alphabet. I do not at all excuse myself for this foolish neglect of opportunities procured at so much expense by my parents and brought so forcibly to my attention by my Preceptors. Perhaps if I had been introduced to the ancients through their history and customs, instead of through their grammar and syntax, I might have had a better record.
}
"""

# combine block_1 and block_2
context = block_1 + "---\n" + block_2

sys_prompt = f"""
Instructions:
- Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
- Utilize the context provided for accurate and specific information.
- Incorporate your preexisting knowledge to enhance the depth and relevance of your response.
Context: 
"""

sys_prompt


"\nInstructions:\n- Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'\n- Utilize the context provided for accurate and specific information.\n- Incorporate your preexisting knowledge to enhance the depth and relevance of your response.\nContext: \n{\ntitle: The Autobiography of Benjamin Franklin - Benjamin Franklin\nauthor: Benjamin Franklin\ntext: That felicity, when I reflected on it, has induced me sometimes to say, that were it offered to my choice, I should have no objection to a repetition of the same life from its beginning, only asking the advantages authors have in a second edition to correct some faults of the first. So I might, besides correcting the faults, change some sinister accidents and events of it for others more favorable. But though this were denied, I should still accept the offer. Since such a repetition is not to be expected, the next thing most like living one’s life over again seems to be a recollection of that life, a

In [14]:
import os

from groq import Groq

client = Groq(
    # This is the default and can be omitted
    api_key=os.environ.get("GROQ_API_KEY"),
)

user_query = "what is the meaning of life you consult people who are provided in the context?"

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": sys_prompt,
        },
        {
            "role": "user",
            "content": user_query,
        }
    ],
    model="llama3-70b-8192",
)

response = chat_completion.choices[0].message.content

# split the response by 150 characters
response = [response[i:i+150] for i in range(0, len(response), 150)]






In [15]:
# print the response array
for i in response:
    print(i)

An intriguing question!

Let's consult Benjamin Franklin and Winston Churchill, both prominent figures in history, to gain insights into their perspec
tives on the meaning of life.

Benjamin Franklin's autobiography suggests that the meaning of life lies in living it to the fullest, with the ability 
to reflect on one's experiences and learn from them. He implies that if he could relive his life, he would make corrections and changes to overcome pa
st mistakes and unfavorable events. However, even without the opportunity to relive his life, he finds value in recollecting and recording his experie
nces, making them as durable as possible through writing. This implies that Franklin believes the meaning of life comes from living, learning, and sha
ring one's experiences with others.

Winston Churchill, on the other hand, provides a slightly different perspective. In his writing, he highlights th
e importance of passion, interest, and engagement in shaping one's life. He admits that he was

In [52]:
from sklearn.metrics.pairwise import cosine_similarity


embedding_model = TextEmbedding()

count = 0
extra_context = ""
sys_prompt = ""
chat_history = []

while True:
  # Get user input from the console
  user_input = input("You: ")
  if count == 0:
    print("User input: ", user_input)
    query_embedding = list(embedding_model.embed(user_input))
    # Join the blocks embeddings with the pages embeddings
    similarities = cosine_similarity(query_embedding, combined_embeddings)
    top_data = find_most_similar_embeddings(similarities, blocks, pages, blocks_length, pages_length)
    titles = [item[3] for item in top_data]
    print("Titles of the data:")
    for title in titles:
        print(title)
    extra_context = " ".join([f"{item[1]} - {item[2]} - {item[3]}" for item in top_data])
    sys_prompt = f"""
      Instructions:
      - Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
      - Utilize the context provided for accurate and specific information.
      - Incorporate your preexisting knowledge to enhance the depth and relevance of your response.
      Context: {extra_context}
      """
    # Initialize the chat history
    chat_history.append({
      "role": "system",
      "content": sys_prompt,
    })


  print("You:", user_input)

  # check if input contains exit
  if user_input.find("exit") != -1:
    break

  # Append the user input to the chat history
  chat_history.append({"role": "user", "content": user_input})

  response = client.chat.completions.create(model="llama3-70b-8192",
                                            messages=chat_history,
                                            max_tokens=500
                                            )
  # Append the response to the chat history
  chat_history.append({
      "role": "assistant",
      "content": response.choices[0].message.content
  })
  # Print the response
  print("Assistant:", response.choices[0].message.content)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

User input:  What is the purpose of life?
Titles of the data:
/Users/freddycaceres/Research/Second brain/2022_12_25.md#9
/Users/freddycaceres/Research/Second brain/2023_05_13.md#3
/Users/freddycaceres/Research/Second brain/My Early Life - Winston S. Churchill, William Manchester (Introduction).md#130
/Users/freddycaceres/Research/Second brain/Rhetoric - Aristotle.md#27
/Users/freddycaceres/Research/Second brain/The Fastest Way to Kill... - @Austen on Twitter.md#5
/Users/freddycaceres/Research/Second brain/Tweets From Figen - @TheFigen on Twitter.md#7
/Users/freddycaceres/Research/Second brain/2024_07_09.md#19
/Users/freddycaceres/Research/Second brain/to___study.md#124
/Users/freddycaceres/Research/Second brain/to___study.md#122
/Users/freddycaceres/Research/Second brain/My Early Life - Winston S. Churchill, William Manchester (Introduction).md#131
You: What is the purpose of life?
Assistant: According to the context provided, the purpose of life is to "glorify God and find joy in him.